# 1. Imports and Paths

In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path
from common_types import FolderHierarchy
from utils import extract_file_from_files
from dataset_preparation import get_folder_hierarchy
from typing import Callable, Dict, Optional, Tuple, List

pd.set_option("display.float_format", lambda x: "%.3f" % x)
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Times New Roman"
plots_dir = Path("/home/aai30/nfs-share/b_hfl/flower/baselines/b_hfl/b_hfl/plots")
tables_dir = Path("/home/aai30/nfs-share/b_hfl/flower/baselines/b_hfl/b_hfl/tables")


# 2. Setup Hydra

In [14]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from hydra.utils import call
from omegaconf import OmegaConf
from hydra_utils import build_hydra_client_fn_and_recursive_builder_generator


In [5]:
initialize(version_base=None, config_path="conf/")


hydra.initialize()

In [11]:
cfg = compose(config_name="config.yaml")


# 3. Plotting Loss landscapes

In [ ]:
top_models_path: Path = Path(
    "/home/aai30/nfs-share/b_hfl/flower/baselines/b_hfl/b_hfl/outputs/2023-08-17/08-34-16/client_data"
)
target_models_path: Path = Path(
    "/home/aai30/nfs-share/b_hfl/flower/baselines/b_hfl/b_hfl/outputs/2023-08-17/08-46-36"
)
bottom_models_path: Path = Path(
    "/home/aai30/nfs-share/b_hfl/flower/baselines/b_hfl/b_hfl/outputs/2023-08-17/08-52-20/client_data"
)


In [ ]:


from common_types import ClientFN, RecursiveBuilder
from flwr.common import NDArrays


def evaluate_model_at_path(path_dict: FolderHierarchy) -> Optional[Tuple[float, Dict]]:
    (
        client_fn,
        get_client_recursive_builder,
        _,
        on_evaluate_config_function,
        load_parameters_file,
        _,
    ) = build_hydra_client_fn_and_recursive_builder_generator(cfg, path_dict)

    evaluate_fn: Callable[[int, NDArrays, Dict], Optional[Tuple[float, Dict]]] = call(
        cfg.fed.get_fed_eval_fn,
        root_path=path_dict["path"],
        client_fn=client_fn,
        recursive_builder=get_client_recursive_builder(path_dict),
        on_evaluate_config_function=on_evaluate_config_function,
    )
    parameters_file: Optional[Path] = extract_file_from_files(path_dict["path"], "parameters")
    if parameters_file is None:
        raise ValueError(f"Could not find parameters file in {path_dict['path']}")
    parameters: NDArrays = load_parameters_file(parameters_file)
    return evaluate_fn(0, parameters, {})


In [ ]:
from tkinter import N
from common_types import ParametersLoader


class FileHierarchy(FolderHierarchy):
    files: List[Path]
    target_parameters: NDArrays
    top_parameters: NDArrays
    bottom_paramters: NDArrays


def plot_loss_landscape(
    target_model_path: Path, top_model_path: Path, bottom_model_path: Path
):
    target_path_dict: FolderHierarchy = get_folder_hierarchy(target_model_path)
    top_path_dict: FolderHierarchy = get_folder_hierarchy(top_model_path)
    bottom_path_dict: FolderHierarchy = get_folder_hierarchy(bottom_model_path)
    if not check_compatibility(
        target_path_dict, top_path_dict, bottom_path_dict
    ):
        raise ValueError("Folder structures are not compatible.")
    
    
    
    
